In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

In [27]:
df = pd.read_csv('team_df.csv')
df = df.drop(columns='team')
df.head()

,score_team,score_opp,map_pick,rating,acs,kill,death,assist,kast%,adr,...,opponent_RRQ,opponent_SEN,opponent_SPB,opponent_T1,opponent_TH,opponent_TL,opponent_TLN,opponent_TS,opponent_VIT,opponent_ZETA
0,13.0,3.0,1.0,1.306,238.0,14.6,9.8,7.6,86.4,153.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,13.0,0.0,0.672,182.0,9.8,14.6,2.6,62.8,123.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13.0,11.0,0.0,1.082,191.4,16.6,14.0,8.2,77.6,121.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.0,13.0,1.0,0.952,175.6,14.0,16.6,5.8,68.6,115.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.0,7.0,0.0,1.168,202.4,14.4,12.6,7.2,68.0,130.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:

train, test = train_test_split(df, train_size=0.75)
print(f'Train size: {train.shape} \nTest size: {test.shape}')

Train size: (934, 58) 
Test size: (312, 58)


In [29]:
x_train = train.drop(columns=['team_win','score_team','score_opp'])
x_test = test.drop(columns=['team_win','score_team','score_opp'])
y_train = train['team_win']
y_test = test['team_win']

## Simple Naive Bayes Classification

In [30]:
NB_model = GaussianNB()
NB_model.fit(x_train, y_train)
y_pred = NB_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.65      0.74      0.69       148
         1.0       0.73      0.65      0.69       164

    accuracy                           0.69       312
   macro avg       0.69      0.69      0.69       312
weighted avg       0.69      0.69      0.69       312



In [31]:
print(f'Overall accuracy: {accuracy_score(y_pred,y_test)}')

Overall accuracy: 0.6891025641025641


## Support Vector Classifier

In [32]:
# Linear SVC
lin_SVC = SVC(kernel='linear')
lin_SVC.fit(x_train, y_train)
y_pred = lin_SVC.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       148
         1.0       0.91      0.91      0.91       164

    accuracy                           0.91       312
   macro avg       0.91      0.91      0.91       312
weighted avg       0.91      0.91      0.91       312



In [33]:
# rbf kernel
rbf_SVC = SVC(kernel='rbf')
rbf_SVC.fit(x_train, y_train)
y_pred = rbf_SVC.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.88      0.80      0.84       148
         1.0       0.84      0.90      0.87       164

    accuracy                           0.86       312
   macro avg       0.86      0.85      0.85       312
weighted avg       0.86      0.86      0.86       312



## Artificial Neural Network

In [42]:
nn_model = Sequential()
nn_model.add(Dense(units = 12, input_dim = x_train.shape[1], activation = 'softmax'))
nn_model.add(Dense(units = 24, activation = 'relu'))
nn_model.add(Dense(units = 15, activation = 'relu'))
nn_model.add(Dense(units = 1, activation='sigmoid'))

c:\Users\Chris Mo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
opt = SGD(learning_rate=0.02)
nn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
nn_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 12)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 15)             │           375 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,375 (5.37 KB)

 Trainable params: 1,375 (5.37 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
nn_model.fit(x_train,y_train, epochs=100)

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.4037 - loss: 0.6962 
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.5001 - loss: 0.6932
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - accuracy: 0.5043 - loss: 0.6932
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.4947 - loss: 0.6936
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5302 - loss: 0.6922 
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4812 - loss: 0.6934 
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4870 - loss: 0.6940 
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.5240 - loss: 0.6925
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.4749 - loss: 0.6936
Epoch 10/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5137 - loss: 0.6931 
Epoch 11/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - accuracy: 0.4965 - loss: 0.6937
Epoch 12/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9

In [47]:
y_pred = nn_model.predict(x_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [48]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.47      0.99      0.64       148
         1.0       0.50      0.01      0.02       164

    accuracy                           0.47       312
   macro avg       0.49      0.50      0.33       312
weighted avg       0.49      0.47      0.32       312



## Hyperparameter Tuning